# Game Generation Research Notebook

The purpose of this notebook is to provide an overview of my research into game theory, specifically in the application of LLMs for dynamically generating games. This research project will focus on sequential games, though other games may be explored later.

In [20]:
import requests
from typing import Tuple
import json

def get_service() -> Tuple[str, str]:
        """
        Loads the credentials for the service.

        Returns:
            token: token to use for authentication.
            svc_url: url of the service instance.
        """
        credentials_path = "./credentials.json"
        try:
            with open(credentials_path, "r") as file:
                svc_key = json.loads(file.read())
        except Exception as error:
            print(f"Could not load the credentials: {error}")

        return requests.post(
            f"{svc_key['uaa']['url']}/oauth/token",
            auth=(svc_key["uaa"]["clientid"], svc_key["uaa"]["clientsecret"]),
            params={"grant_type": "client_credentials"}
        ).json()["access_token"], svc_key["url"]

def prompt_llm(prior, prompt, contexts=[], temp=0.7) -> str:
        """
        Get the completions from the LLM.
        """
        if prompt == "": return ""
        token, svc_url = get_service()
        edited_prompt = prompt
        messages = [{'role': 'assistant', 'content': context} for context in contexts]
        messages.append({"role": "user", "content": edited_prompt})
        messages.insert(0, {"role": "system", "content": prior})
        response = requests.post(
            f"{svc_url}/api/v1/completions",
            headers={
                "Authorization": f"Bearer {token}",
                "Content-Type": "application/json"
            },
            
            json={
                "deployment_id": "gpt-4-32k",
                "messages": messages,
                "max_tokens": 500,
                "temperature": temp,
                "frequency_penalty": 0,
                "presence_penalty": 0,
                "top_p": 0.95,
                "stop": "null"
            }
        )

        if response.status_code != 200:
            print(f"Error: {response.text}")
            return False, ""
        return str(response.json()["choices"][0]["message"]["content"])

## Dynamic Code Generation & Execution

The following is a demonstration of how to dynamically execute code in Python. This will be useful for generating sequential games.

In [21]:
def extract_code(response):
    code = response.split("```")
    if len(code) > 1:
        return code[1][len("python"):]
    return response

def generate_code(prompt, contexts=[], temp=0.2):
    response = prompt_llm(
        prior="You are a coding assistant. You are helping a user write a Python program. Only respond with relevant code; no explanation or comments.",
        prompt=prompt,
        contexts=contexts,
        temp=temp
    )
    return extract_code(response)

In [22]:
code = generate_code("Write code for a Python program that prints the first 10 twin prime numbers.")
print(code)

def is_prime(n):
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

def twin_primes(n):
    primes = []
    i = 2
    while len(primes) < n:
        if is_prime(i) and is_prime(i + 2):
            primes.append((i, i + 2))
        i += 1
    return primes

for pair in twin_primes(10):
    print(pair)


In [23]:
exec(code)

(3, 5)
(5, 7)
(11, 13)
(17, 19)
(29, 31)
(41, 43)
(59, 61)
(71, 73)
(101, 103)
(107, 109)


## Sequential Games

Sequential games are a type of game where players take turns to make decisions. The decisions made by each player can be based on the decisions made by the previous players. Sequential games are often represented as trees, where the nodes represent decision points and the edges represent the possible decisions that can be made.

We will be using this type of game as a structure for the LLM to generate games.

### Generating Features

The first step in generating a sequential game is to generate the features that will be used to represent the game. These features will generated by the LLM to generate the game.


### Model Assumptions

- N players
- Objects are given to the model

In [24]:
def generate_game(prompt):
    prior = '''
    Your job will be to look for a fun way to combine real-world objects to make a sequential game (game theory). Your response should be formatted as:

    Representative Emoji: eg. 🎲

    Game Title: eg. Dice Roll

    Set-Up Instructions: eg.
    1. Place the dice in the center of the table.
    2. Each player takes turns rolling the dice.

    Turn-Instructions: eg.
    1. Player rolls the dice.
    2. Player reads the number at the top of the dice.

    Winning/Losing Condition: eg.
    1. The player who rolls the highest number wins.
    2. The player who rolls the lowest number loses.

    Ignored-Objects: eg.
    1. Coin
    2. Chess Board

    -------------------

    Your response should have NO ambiguity. You also do not need to use all objects (IGNORE IRRELEVANT SURFACES AND FURNITURE).
    '''
    response = prompt_llm(
        prior=prior,
        prompt=prompt,
    )
    return response

### Eliminating Irrelevant Features

For now, we will simply prompt the LLM to ignore objects that it thinks do not matter. In the future this can be done by a classifier.

In [25]:
N = 10
objects = ["charging cable", "eggs", "laptop", "basket", "keyboard"]
quantities = [1, 12, 1, 3, 1]
prompt = ', '.join([f"{quant} {obj}s" if quant > 1 else obj for obj, quant in zip(objects, quantities)]) + f" {N} players"
game_response = generate_game(prompt)
print(game_response)

Representative Emoji: 🥚

Game Title: Egg Transfer Race

Set-Up Instructions: 
1. Place the 12 eggs in one basket.
2. Put the other two baskets at a distance of 5-10 steps (depending on space) on either side of the basket with the eggs.
3. Place the laptop and keyboard at a convenient viewing point.

Turn-Instructions: 
1. The first player starts at the basket with the eggs.
2. The player must pick up an egg from the basket, carefully wrap it in the charging cable, and then carry it to one of the other baskets.
3. The player must then return to the original basket and repeat the process until all eggs have been moved.
4. This process is timed using the stopwatch function on the laptop.
5. The next player then repeats the process.

Winning/Losing Condition: 
1. The player who transfers all the eggs in the shortest time is the winner.
2. The player who takes the longest time loses.
3. If a player drops and breaks an egg, they automatically lose.

Ignored-Objects: 
1. Keyboard


In [26]:
def numbered_list_to_array(text: str):
    return [line[line.index('.') + 2:] for line in text.split('\n') if line.strip() != ""]

In [27]:
import regex as re
structured_response = [line.split(':') for line in game_response.split("\n\n")]
# turn the response into a dictionary
meta_game = {
    key: numbered_list_to_array(value) if re.match(r'^\s*\d+\.\s+.+', value) else value.strip() 
    for key, value in structured_response
}
meta_game

{'Representative Emoji': '🥚',
 'Game Title': 'Egg Transfer Race',
 'Set-Up Instructions': ['Place the 12 eggs in one basket.',
  'Put the other two baskets at a distance of 5-10 steps (depending on space) on either side of the basket with the eggs.',
  'Place the laptop and keyboard at a convenient viewing point.'],
 'Turn-Instructions': ['The first player starts at the basket with the eggs.',
  'The player must pick up an egg from the basket, carefully wrap it in the charging cable, and then carry it to one of the other baskets.',
  'The player must then return to the original basket and repeat the process until all eggs have been moved.',
  'This process is timed using the stopwatch function on the laptop.',
  'The next player then repeats the process.'],
 'Winning/Losing Condition': ['The player who transfers all the eggs in the shortest time is the winner.',
  'The player who takes the longest time loses.',
  'If a player drops and breaks an egg, they automatically lose.'],
 'Ignor

In [28]:
formatted_ignored_objects = [obj.lower() for obj in meta_game.get("Ignored-Objects", [])]
used_objects = [obj for obj in objects if obj not in formatted_ignored_objects]
used_objects

['charging cable', 'eggs', 'laptop', 'basket']

In [29]:
actions_payoffs_resp = prompt_llm(
    "You are building a sequential game in game theory. Answer with just the actions that a player can do on their turn and their payoffs in a numbered list. Answer ONLY with the numbered list in the form: 1. Action, Payoff (eg. +3)",
    f'''Here are the objects in the game: {used_objects}. MAKE SURE YOUR ACTIONS ARE MUTUALLY INDEPENDENT''',
    contexts=[game_response],
    temp=0.3
)

print(actions_payoffs_resp)

1. Pick up egg, Payoff +1
2. Wrap egg in charging cable, Payoff +1
3. Carry egg to another basket, Payoff +1
4. Return to original basket, Payoff +1
5. Drop and break egg, Payoff -5
6. Use laptop to time, Payoff +1
7. Finish transferring all eggs, Payoff +5
8. Take longest time, Payoff -3


In [30]:
actions_payoffs = [action_payoff.split(", ") for action_payoff in numbered_list_to_array(actions_payoffs_resp)]
actions = [action for action, _ in actions_payoffs]

In [31]:
game_code = generate_code(
    f"Write a Python script that models the state of the game (with a class called 'Game') with 2 players and has a function called 'step' that takes in an action from {actions} and steps the state of the game; and a function called 'get_winner' that returns the winner of the game (integer).",
    contexts=[game_response, f"Actions and Payoffs:\n{actions_payoffs_resp}"],
)


print(game_code)


class Game:
    def __init__(self):
        self.players = [0, 0]
        self.actions = {
            'Pick up egg': 1,
            'Wrap egg in charging cable': 1,
            'Carry egg to another basket': 1,
            'Return to original basket': 1,
            'Drop and break egg': -5,
            'Use laptop to time': 1,
            'Finish transferring all eggs': 5,
            'Take longest time': -3
        }
        self.current_player = 0

    def step(self, action):
        self.players[self.current_player] += self.actions[action]
        self.current_player = 1 - self.current_player

    def get_winner(self):
        return self.players.index(max(self.players))



In [32]:
import random

game_actions = [random.choice(actions) for _ in range(10)]
game_actions

['Use laptop to time',
 'Take longest time',
 'Take longest time',
 'Carry egg to another basket',
 'Pick up egg',
 'Carry egg to another basket',
 'Return to original basket',
 'Finish transferring all eggs',
 'Pick up egg',
 'Take longest time']

In [33]:
exec(game_code)

game = Game()
for action in game_actions:
    game.step(action)
    print(game.get_winner())
game.get_winner()

0
0
0
0
0
0
0
1
1
0


0

## Game Generation

With all the pieces in place, we can now make a function to generate a game given a set of objects.

In [34]:
def generate_game_function(objects: list[str], quantities: list[int]) -> Tuple[str, str]:
    '''
    @param objects: list of objects to be used in the game
    @return: string representing the python function to run the game, string representing the game response
    '''
    if len(objects) != len(quantities):
        raise ValueError("The number of objects and quantities must be the same")
    prompt = ', '.join([f"{quant} {obj}s" if quant > 1 else obj for obj, quant in zip(objects, quantities)])
    game_response = generate_game(prompt)
    print(game_response)
    structured_response = [line.split(':') for line in game_response.split("\n\n")]
    # turn the response into a dictionary
    meta_game = {
        key: numbered_list_to_array(value) if re.match(r'^\s*\d+\.\s+.+', value) else value.strip() 
        for key, value in structured_response
    }
    formatted_ignored_objects = [obj.lower() for obj in meta_game.get("Ignored-Objects", [])]
    used_objects = [obj for obj in objects if obj not in formatted_ignored_objects]
    actions_payoffs_resp = prompt_llm(
        "You are building a sequential game in game theory. Answer with just the actions that a player can do on their turn and their payoffs in a numbered list. Answer ONLY with the numbered list in the form: 1. Action, Payoff (eg. +3)",
        f'''Here are the objects in the game: {used_objects}. MAKE SURE YOUR ACTIONS ARE MUTUALLY INDEPENDENT''',
        contexts=[game_response],
        temp=0.3
    )
    actions_payoffs = [action_payoff.split(", ") for action_payoff in numbered_list_to_array(actions_payoffs_resp)]
    actions = [action for action, _ in actions_payoffs]
    game_code = generate_code(
        f"Write a Python function that takes as input an array of action strings and number of players and returns the winner.",
        contexts=[game_response, f"Actions and Payoffs:\n{actions_payoffs_resp}"],
    )
    return game_code, game_response

## Reinforcement Learning

Now that the game is generated. We can use reinforcement methods to make the game more interesting. This can be done by training a model to play the game and then using the model to generate new games.

In [36]:
actions_probs = prompt_llm(
    prior=game_response,
    prompt="Give me a list of actions and the probability of their completion, format ur answer like [(action, probability[0-1])]"
)